In [1]:
import pandas as pd

# CSV faylni yuklash
df = pd.read_csv("addresses.csv")

# Longitude va Latitude qiymati bo'sh bo'lgan qatorlarni o‘chirish
df_cleaned = df.dropna(subset=['latitude', 'longitude'])

# Tozalangan ma'lumotlarni yangi CSV fayl sifatida saqlash
df_cleaned.to_csv("need_to_be_cetring_file.csv", index=False)

print("Tozalash tugadi. Natija 'cleaned_file.csv' fayliga saqlandi.")


Tozalash tugadi. Natija 'cleaned_file.csv' fayliga saqlandi.


In [2]:
import pandas as pd

# CSV faylni yuklash
df = pd.read_csv("need_to_be_cetring_file.csv")

# Duplikat qatorlarni yo‘qotish
df_cleaned = df.drop_duplicates(subset=["street"], keep="first")

# Tozalangan CSV faylni saqlash
df_cleaned.to_csv("need_to_be_cetring_file.csv", index=False)

print("✅ Duplikatlar olib tashlandi. Natija 'clean_data.csv' fayliga saqlandi.")

✅ Duplikatlar olib tashlandi. Natija 'clean_data.csv' fayliga saqlandi.


In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

# CSV faylni yuklash
df = pd.read_csv("need_to_be_cetring_file.csv")

# O'zbekiston viloyat markazlari koordinatalari
city_centers = {
    "Tashkent": (41.2995, 69.2401),
    "Samarkand": (39.6542, 66.9597),
    "Bukhara": (39.7670, 64.4556),
    "Fergana": (40.3894, 71.7864),
    "Namangan": (40.9983, 71.6726),
    "Andijan": (40.7833, 72.3500),
    "Nukus": (42.4531, 59.6103),
    "Khiva": (41.3786, 60.3639),
    "Urgench": (41.5500, 60.6333),
    "Termez": (37.2242, 67.2783),
    "Jizzakh": (40.1158, 67.8422),
    "Gulistan": (40.4958, 68.7842),
    "Navoi": (40.0844, 65.3792),
    "Zarafshan": (41.5789, 64.1975),
    "Qarshi": (38.8600, 65.8000)
}

# Geolocator yaratish
geolocator = Nominatim(user_agent="geoapiExercises")

# City nomini topish funksiyasi
def find_city(row):
    if pd.notna(row.get("region")):  # Agar shahar nomi allaqachon bo'lsa, o'zgartirmaymiz
        return row["region"]
    
    try:
        location = geolocator.reverse((row["latitude"], row["longitude"]), exactly_one=True)
        if location:
            return location.raw.get("address", {}).get("city", "Unknown")  # Shahar nomini qaytaradi
    except:
        pass
    
    # Agar geopy ishlamasa, eng yaqin viloyatni topamiz
    min_dist = float("inf")
    nearest_city = "Unknown"
    
    for city, coords in city_centers.items():
        dist = geodesic(coords, (row["latitude"], row["longitude"])).km
        if dist < min_dist:
            min_dist = dist
            nearest_city = city
    
    return nearest_city

# Har bir joy uchun viloyat markazini aniqlash
df["region"] = df.apply(find_city, axis=1)

# Masofani hisoblash
def calculate_distance(row):
    city = row["region"]
    if city in city_centers:
        city_center_coords = city_centers[city]
        point_coords = (row["latitude"], row["longitude"])
        return geodesic(city_center_coords, point_coords).km
    return "Unknown"

# Masofani qo'shish
df["distance_from_center"] = df.apply(calculate_distance, axis=1)

# Natijani saqlash
df.to_csv("cleaned_with_city_and_distance.csv", index=False)

print("✅ City topildi va masofalar hisoblandi. 'cleaned_with_city_and_distance.csv' yaratildi.")


✅ City topildi va masofalar hisoblandi. 'cleaned_with_city_and_distance.csv' yaratildi.
